In [391]:
import pysam
import pybedtools
import pandas as pd
import clipper.src.CLIP_analysis
from collections import OrderedDict
from scipy.stats import hypergeom
import numpy as np
from collections import defaultdict
from os.path import basename
from os import listdir
from scipy.stats import norm

In [383]:
p_value_df = pd.read_excel("/home/jnussbac/projects/encode_m6a/analysis/encode_m6a_v1/p_value_df.xls", sheetname="all")
path = "/home/elvannostrand/data/clip/CLIPseq_analysis/ENCODE_v8_072215/"

m6a_peaks_bed = "/home/jnussbac/projects/encode_m6a/analysis/encode_m6a_v1/hepg2_replicate1.peaks.bed"
m6a_filename = basename(m6a_peaks_bed)
m6a_pybedtool = pybedtools.BedTool(m6a_peaks_bed)

from clipper.src import CLIP_analysis
assigned_regions, regions = CLIP_analysis.regions_generator()

In [384]:
cluster_regions_m6a = CLIP_analysis.assign_to_regions(m6a_pybedtool, "m6a_pybedtool", assigned_regions, 
                                                       "assigned", nrand=10, species="hg19")

Not Valid bed12 file, continuing processing, some things may be strange
There are a total 17189 clusters I'll examine
For region: cds found 6553 that overlap and 10636 that don't
Not Valid bed12 file, continuing processing, some things may be strange
For region: three_prime_utrs found 2743 that overlap and 7893 that don't
Not Valid bed12 file, continuing processing, some things may be strange
For region: five_prime_utrs found 1917 that overlap and 5976 that don't
Not Valid bed12 file, continuing processing, some things may be strange
For region: proxintron500 found 1348 that overlap and 4628 that don't
Not Valid bed12 file, continuing processing, some things may be strange
For region: distintron500 found 2425 that overlap and 2203 that don't
Not Valid bed12 file, continuing processing, some things may be strange
After assigning 2203 un-categorized regions


## loop through analysis of all RBPs

In [386]:
rbp_list = []
for file in listdir(path):
    if file.endswith(".peaks.bed"):
        rbp_list.append(file)

In [409]:
p_value_df = pd.read_excel("/home/jnussbac/projects/encode_m6a/analysis/encode_m6a_v1/p_value_df.xls", sheetname="all")

IOError: [Errno 24] Too many open files: '/home/jnussbac/projects/encode_m6a/analysis/encode_m6a_v1/p_value_df.xls'

In [399]:
for rbp in rbp_list:
    #assign rbp_peaks_bed to the peaks.bed file
    rbp_peaks_bed = path + rbp
    rbp_filename = basename(rbp_peaks_bed)

    rbp_pybedtool = pybedtools.BedTool(rbp_peaks_bed)

    cluster_regions_rbp = CLIP_analysis.assign_to_regions(rbp_pybedtool, "rbp_pybedtool", assigned_regions, "assigned", nrand=10, species="hg19")

    rbp_intersects_background = []
    for name, bedtool in cluster_regions_rbp["all"]["rand"].items():
        rbp_intersects_background.append(len(bedtool.intersect(cluster_regions_rbp["all"]["real"], u=True)))

    real_vs_real = len(cluster_regions_m6a["all"]["real"].intersect(cluster_regions_rbp["all"]["real"], u=True))

    z_all = np.abs((real_vs_real - np.mean(rbp_intersects_background)) / np.std(rbp_intersects_background))
    p_value_df[m6a_filename][rbp_filename] = norm.sf(z_all)*2 #multiply by 2 for 2-tailed t-test

    p_value_df.to_excel("/home/jnussbac/projects/encode_m6a/analysis/encode_m6a_v1/p_value_df.xls", sheet_name="all")

Not Valid bed12 file, continuing processing, some things may be strange, also this will cause a file leak in pybedtools, watch out


BedToolsFileError: Error opening file

In [400]:
# rbp_peaks_bed = path + "271_01_HNRNPC.merged.r2.peaks.bed"
# rbp_filename = basename(rbp_peaks_bed)
# print rbp_filename

In [401]:
# rbp_pybedtool = pybedtools.BedTool(rbp_peaks_bed)

In [402]:
# cluster_regions_rbp = CLIP_analysis.assign_to_regions(rbp_pybedtool, "rbp_pybedtool", assigned_regions, 
#                                                        "assigned", nrand=10, species="hg19")

In [403]:
# rbp_intersects_background = []
# for name, bedtool in cluster_regions_rbp["all"]["rand"].items():
#     rbp_intersects_background.append(len(bedtool.intersect(cluster_regions_rbp["all"]["real"], u=True)))
    # print rbp_intersects_background

In [404]:
# real_vs_real = len(cluster_regions_m6a["all"]["real"].intersect(cluster_regions_rbp["all"]["real"], u=True))

In [371]:
# rbp_list[0:4]

In [372]:
# p_value_df = pd.read_excel("/home/jnussbac/projects/encode_m6a/analysis/encode_m6a_v1/p_value_df.xls", sheetname="all")
# p_value_df = pd.DataFrame(index=rbp_list, columns=["hepg2_replicate1.peaks.bed", 
#                       "hepg2_replicate2.peaks.bed", 
#                       "hepg2_replicate3.peaks.bed", 
#                       "k562_replicate1.peaks.bed", 
#                       "k562_replicate2.peaks.bed", 
#                       "k562_replicate3.peaks.bed"])
# p_value_df = p_value_df.astype(float)


In [405]:
#calculate the z score
# z_all = np.abs((real_vs_real - np.mean(rbp_intersects_background)) / np.std(rbp_intersects_background))

In [375]:
#calculate the p-value using the pdf
# p_value_df[m6a_filename][rbp_filename] = norm.sf(z_all)*2 #multiply by 2 for 2-tailed t-test

In [394]:
p_value_df.head(50)

,hepg2_replicate1.peaks.bed,hepg2_replicate2.peaks.bed,hepg2_replicate3.peaks.bed,k562_replicate1.peaks.bed,k562_replicate2.peaks.bed,k562_replicate3.peaks.bed
327_01_RBM34.merged.r2.peaks.bed,1.346729e-248,6.323648e-255,4.440414e-204,0,0,0
241_01_DDX42.merged.r2.peaks.bed,9.567489e-116,NaN,NaN,NaN,NaN,NaN
223_01_FKBP4.merged.r2.peaks.bed,0.000000e+00,NaN,NaN,NaN,NaN,NaN
F3_IP_mnCLIP_A2B1_med_1_hnRNPA2B1.merged.r2.peaks.bed,7.912572e-85,NaN,NaN,NaN,NaN,NaN
272_02_U2AF2.merged.r2.peaks.bed,6.970073e-255,NaN,NaN,NaN,NaN,NaN
266_01_FKBP4.merged.r2.peaks.bed,0.000000e+00,NaN,NaN,NaN,NaN,NaN
237_01_FXR2.merged.r2.peaks.bed,0.000000e+00,NaN,NaN,NaN,NaN,NaN
235_01_XRN2.merged.r2.peaks.bed,0.000000e+00,NaN,NaN,NaN,NaN,NaN
249_02_HNRNPU.merged.r2.peaks.bed,1.678356e-140,NaN,NaN,NaN,NaN,NaN
213_02_RBM34.merged.r2.peaks.bed,1.548837e-196,NaN,NaN,NaN,NaN,NaN


In [406]:
# p_value_df.to_excel("/home/jnussbac/projects/encode_m6a/analysis/encode_m6a_v1/p_value_df.xls", sheet_name="all")